In [5]:
import data_api3.reader as reader
import pandas as pd

In [19]:
query = {
  "channels": ["MHC1:IST:2"],
  "range": {
    "type": "date",
    "startDate": "2019-11-05T10:00:00.000000000Z",
    "endDate": "2019-11-06T10:10:00.000000000Z"
  }
}


25, 25


In [23]:
%timeit data = reader.request(query, url="http://hipa-daq-01:8090/api/v1/query")

25, 25
25, 25
25, 25
25, 25
25, 25
25, 25
25, 25
25, 25
23.7 s ± 2.76 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
df = pd.DataFrame(data["MHC1:IST:2"])

In [21]:
df.head()

,timestamp,pulse_id,MHC1:IST:2
0,1572949110807279570,-1,1.1
1,1572949111007279570,-1,25.8
2,1572949111207279570,-1,41.3
3,1572949111407279570,-1,43.7
4,1572949111607279570,-1,46.9


In [22]:
len(df)

404738